### Scraping Twitter Network Tweets
https://github.com/Jefferson-Henrique/GetOldTweets-python

In [1]:
import pandas as pd
import got3
from tqdm import tqdm_notebook

import os
os.getcwd()

'C:\\Users\\pparker.KLASJ\\Desktop\\tscrape\\GetOldTweets-python'

In [2]:
# scrape a tweet sample
tweetCriteria = got3.manager.TweetCriteria().setUsername('barackobama').setMaxTweets(1)
tweet = got3.manager.TweetManager.getTweets(tweetCriteria)[0]
 
print(tweet.text)

Notre Dame is one of the world’s great treasures, and we’re thinking of the people of France in your time of grief. It’s in our nature to mourn when we see history lost – but it’s also in our nature to rebuild for tomorrow, as strong as we can.pic.twitter.com/SpMEvv1BzB


In [3]:
# assign twitter account for network leader
leader = "Cointelegraph"

In [4]:
# configure the scraper
tweetCriteria = got3.manager.TweetCriteria().setUsername(leader).setSince("2019-04-12").setUntil("2019-04-16").setMaxTweets(2000)
tweet = got3.manager.TweetManager.getTweets(tweetCriteria)

In [5]:
# format network leader tweets into dataframe
text_list = [i.text for i in tweet]
date_list = [i.date for i in tweet]

df_tweet = pd.DataFrame.from_dict({'date':date_list, 'tweet':text_list, 'id':[leader]*len(date_list)})
df_tweet.head()

,date,tweet,id
0,2019-04-15 15:58:36,ConsenSys is reportedly trying to attract outs...,Cointelegraph
1,2019-04-15 15:11:55,Don't miss the most memorable quotes from last...,Cointelegraph
2,2019-04-15 15:09:47,Portland State University researchers have pre...,Cointelegraph
3,2019-04-15 14:01:08,Reuters: France to ask other EU states to adop...,Cointelegraph
4,2019-04-15 13:11:07,Ethereum engineer Lukas Hohl has joined Swissc...,Cointelegraph


In [6]:
# import the leader's following list
df_tid = pd.read_csv('cointelegraph_following.csv',index_col=False)
df_tid.head()

,Unnamed: 0,following
0,0,binance
1,1,winklevoss
2,2,winklevosscap
3,3,LucreziaCorner
4,4,CointelegraphAr


In [14]:
# In case of time-out restart, find the last following id
#df_tweet = pd.read_csv('df_tweet_r1.csv')
df_tweet.tail(1)['id'].values[0]

'Cointelegraph'

In [22]:
# find the index of the last following
df_tid.loc[df_tid['following']=='winklevosscap']

,Unnamed: 0,following
2,2,winklevosscap


In [23]:
# Scrape the tweets of the leader
for follower in tqdm_notebook(df_tid['following'][76:].tolist()):
    
    try:
        tweetCriteria = got3.manager.TweetCriteria().setUsername(follower).setSince("2019-04-12").setUntil("2019-04-19").setMaxTweets(10000)
        tweet = got3.manager.TweetManager.getTweets(tweetCriteria)
        f_text_list = [i.text for i in tweet]
        f_date_list = [i.date for i in tweet]

        df_follow = pd.DataFrame.from_dict({'date':f_date_list, 'tweet':f_text_list, 'id':[follower]*len(f_date_list)})
        df_tweet = pd.concat([df_tweet, df_follow])

        df_tweet.to_csv('df_tweet_r1.csv',index=False, encoding='utf-8')
        
    except:
        pass
    
df_tweet.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4983 entries, 0 to 13
Data columns (total 3 columns):
date     4983 non-null datetime64[ns]
tweet    4983 non-null object
id       4983 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 155.7+ KB


In [24]:
from langdetect import detect

lang = detect("Ein, zwei, drei, vier")

print(lang)
#output: de

de


In [25]:
# detect languages
lang_list = []

for i,nrows in tqdm_notebook(df_tweet.iterrows()):
    lang = ""
    try:
        lang = detect(nrows['tweet'])
    except:
        pass
    
    if len(lang)>0:
        lang_list.append(lang)
    else:
        lang_list.append(None)
    
df_tweet['lang'] = lang_list
    
df_tweet.head()

,date,tweet,id,lang
0,2019-04-15 15:58:36,ConsenSys is reportedly trying to attract outs...,Cointelegraph,en
1,2019-04-15 15:11:55,Don't miss the most memorable quotes from last...,Cointelegraph,en
2,2019-04-15 15:09:47,Portland State University researchers have pre...,Cointelegraph,en
3,2019-04-15 14:01:08,Reuters: France to ask other EU states to adop...,Cointelegraph,en
4,2019-04-15 13:11:07,Ethereum engineer Lukas Hohl has joined Swissc...,Cointelegraph,en


In [27]:
# get number of non-English tweets
sum([1 for lang in lang_list if lang != 'en'])

410

In [28]:
# export tweets with language tag
df_tweet.to_csv('df_tweet_r2.csv',index=False, encoding='utf-8')